In [ ]:
library(rethinking)

In [ ]:
d = read.table('normal.csv', header=T, sep=',', stringsAsFactors = F)
# first try for asciminib combinations had very large spread
# likely human error during plate setup, so it was discarded
d = d[!(d$drug == 'ASIM' & d$replicate < 3), ]
d = d[!(d$drug == 'ASDA' & d$replicate < 3), ]
str(d)
precis(d)

In [ ]:
# For comparison purposes:
# simulate a combination that is perfectly exclusive/nonexclusive
# and run it through the model as well.
ic1 = 0.02
ic2 = 0.02
m1 = 2
m2 = 2
r1 = 0.5
r2 = 0.5
sigma = 0.02

conc = 1/(2**rep(0:11, times=3))
abs1 = rnorm(mean=0.2 + 0.6/(1 + exp(m1*(log(conc) - log(ic1)))), sd=sigma, n=36)
abs2 = rnorm(mean=0.2 + 0.6/(1 + exp(m2*(log(conc) - log(ic2)))), sd=sigma, n=36)
abs12.ex = rnorm(mean=0.2 + 0.6/(1 + exp(m1*(log(conc*r1/ic1 + conc*r2/ic2)))), sd=sigma, n=36)
abs12.non = rnorm(mean=0.2 + 0.6/(1 + exp(
        m1*(log(conc*r1/ic1 + conc*r2/ic2 + conc*conc*r1*r2/ic1/ic2))
    )), sd=sigma, n=36)

d.new = data.frame(
    'conc' = rep(conc, times=4),
    'abs' = c(abs1, abs2, abs12.ex, abs12.non),
    'drug' = rep(c('S1', 'S2', 'S1S2E', 'S1S2N'), each=36),
    'replicate' = rep(0:2, times=4, each=12)
)

d = rbind(d, d.new)
str(d)

In [ ]:
data = list()
models = list()

for (drug in c("AS", "DA", "IM", "ASDA", "ASIM", "DAIM", "S1", "S2", "S1S2E", "S1S2N")) {
    print(c("Processing", drug))
    
    data[[drug]] = list(
        C = d[d$drug==drug, ]$conc,
        A = standardize(d[d$drug==drug, ]$abs)
    )
    
    str(data[[drug]])

    models[[drug]] = ulam(
        alist(
            A ~ dnorm(u, s),
            u <- a + (b - a)/(1 + exp(m*(log(C) - log(ic)))),
            a ~ dnorm(0, 1),
            b ~ dnorm(0, 1),
            m ~ dlnorm(-0.5, 1),
            ic ~ dlnorm(-2, 1.5),
            s ~ dexp(1)
        ),
        data = data[[drug]],
        chains = 4,
        cores = 4,
        iter = 4000
    )
}

In [ ]:
for (drug in c("AS", "DA", "IM", "ASDA", "ASIM", "DAIM", "S1", "S2", "S1S2E", "S1S2N")) {
    print(drug)
    print(precis(models[[drug]]))
}

In [ ]:
titles = list(
    "AS" = "Asciminib",
    "DA" = "Dasatinib *",
    "IM" = "Imatinib",
    "ASDA" = "Asciminib + Dasatinib **",
    "ASIM" = "Asciminib + Imatinib **",
    "DAIM" = "Dasatinib + Imatinib ***"
)

options(repr.plot.res=480)

# pdf("ic-normal.pdf", width=8, height=6)

par(mfrow=c(2,3))
for (drug in c("AS", "DA", "IM", "ASDA", "ASIM", "DAIM", "S1", "S2", "S1S2E", "S1S2N")) {
# for (drug in c("AS", "DA", "IM", "ASDA", "ASIM", "DAIM")) {
    C = d[d$drug==drug, ]$conc
    A = d[d$drug==drug, ]$abs
    repl = d[d$drug==drug, ]$replicate + 1
    C.axis = sort(C)
    
#     plot(A ~ log(C), xlim=log(range(d$conc)), ylim=c(0, 1.2), col=repl)
    plot(A ~ log(C), xlim=log(range(d$conc)), ylim=c(0, 1.2), 
         ylab="Absorbance (490 nm)", xlab="ln(Concentration) [ln(nM)]",
         main=titles[[drug]])
    
    post = extract.samples(models[[drug]])
    a = post$a*attr(data[[drug]]$A, "scaled:scale") + attr(data[[drug]]$A, "scaled:center")
    b = post$b*attr(data[[drug]]$A, "scaled:scale") + attr(data[[drug]]$A, "scaled:center")
    m = post$m
    ic = post$ic

    lines(mean(a) + (mean(b) - mean(a))/(1 + exp(mean(m)*(log(C.axis) - log(mean(ic))))) ~ log(C.axis))
    
    hpdis = c()
    for (conc in C.axis) {
        hpdis = c(hpdis, HPDI(a + (b - a)/(1 + exp(m*(log(conc) - log(ic)))), prob=0.89))
    }
    hpdis = matrix(hpdis, nrow=2)
    shade(hpdis, log(C.axis))
    
    sim.abs = sim(models[[drug]], data=list(C=C.axis), post = extract.samples(models[[drug]]))
    abs.HPDI = apply(sim.abs, 2, HPDI, prob=0.89)*attr(data[[drug]]$A, "scaled:scale") + attr(data[[drug]]$A, "scaled:center")
    shade(abs.HPDI, log(C.axis))
}

# dev.off()

In [ ]:
# drug ratios for combination treatment
ratios = list(
    "IM" = 6.31,
    "DA" = 0.0130,
    "AS" = 0.0524
)

titles = list(
    "ASDA" = "Asciminib + Dasatinib",
    "ASIM" = "Asciminib + Imatinib",
    "DAIM" = "Dasatinib + Imatinib"
)

options(repr.plot.res=480)


# pdf("normal-ci.pdf", width=6, height=8)

par(mfrow=c(3,2))
for (drug in c('DAIM', 'ASIM', 'ASDA')) {
    drug.a = substr(drug, 1, 2)
    drug.b = substr(drug, 3, 4)
    r.a = ratios[[drug.a]]/(ratios[[drug.a]] + ratios[[drug.b]])
    r.b = ratios[[drug.b]]/(ratios[[drug.a]] + ratios[[drug.b]])
    fa.axis = 1:99/100
#     fa.axis = seq(0.01, 0.99, length.out=41)
    
    means.ex = c()
    hpdis.ex = c()
    means.non = c()
    hpdis.non = c()
    
    for (fa in fa.axis) {
        post.a = extract.samples(models[[drug.a]])
        post.b = extract.samples(models[[drug.b]])
        post.ab = extract.samples(models[[drug]])
        ed.a = post.a$ic*(fa/(1 - fa))**(1/post.a$m)
        ed.b = post.b$ic*(fa/(1 - fa))**(1/post.b$m)
        ed.ab = post.ab$ic*(fa/(1 - fa))**(1/post.ab$m)
        ci.ex = r.a*ed.ab/ed.a + r.b*ed.ab/ed.b
        ci.non = r.a*ed.ab/ed.a + r.b*ed.ab/ed.b + r.a*r.b*ed.ab*ed.ab/ed.a/ed.b
        means.ex = c(means.ex, mean(ci.ex))
        means.non = c(means.non, mean(ci.non))
        hpdis.ex = c(hpdis.ex, HPDI(ci.ex, prob=0.89))
        hpdis.non = c(hpdis.non, HPDI(ci.non, prob=0.89))
    }
    hpdis.ex = matrix(hpdis.ex, nrow=2)
    hpdis.non = matrix(hpdis.non, nrow=2)
    
    str(means.ex)
    str(hpdis.ex)
    
    plot(means.ex ~ fa.axis, ylim=c(0, 4), xlim=c(0, 1), type='l',
         xlab="Fractional inhibition", ylab="CI", main=titles[[drug]],
         cex.lab=1.2, cex.axis=1.2, cex.main=1.2, cex.sub=1.2)
    shade(hpdis.ex, fa.axis)
    abline(h=1)
    
    plot(means.non ~ fa.axis, ylim=c(0, 4), xlim=c(0, 1), type='l',
         xlab="Fractional inhibition", ylab="CI", main="Assuming nonexclusive interaction")
    shade(hpdis.non, fa.axis)
    abline(h=1)
}

# dev.off()

In [ ]:
# combination index plots for simulated data
# do note, that due to the random noise in the fake data
# these might look somewhat different each time.
par(mfrow=c(3,2)) # make plots the same shape
fa.axis = 1:99/100

means.ex.ex = c()
hpdis.ex.ex = c()
means.non.ex = c()
hpdis.non.ex = c()
means.ex.non = c()
hpdis.ex.non = c()
means.non.non = c()
hpdis.non.non = c()

r.a = 0.5
r.b = 0.5

for (fa in fa.axis) {
    post.a = extract.samples(models[['S1']])
    post.b = extract.samples(models[['S2']])
    post.ab.ex = extract.samples(models[['S1S2E']])
    post.ab.non = extract.samples(models[['S1S2N']])
    ed.a = post.a$ic*(fa/(1 - fa))**(1/post.a$m)
    ed.b = post.b$ic*(fa/(1 - fa))**(1/post.b$m)
    ed.ab.ex = post.ab.ex$ic*(fa/(1 - fa))**(1/post.ab.ex$m)
    ed.ab.non = post.ab.non$ic*(fa/(1 - fa))**(1/post.ab.non$m)
    ci.ex.ex = r.a*ed.ab.ex/ed.a + r.b*ed.ab.ex/ed.b
    ci.ex.non = r.a*ed.ab.ex/ed.a + r.b*ed.ab.ex/ed.b + r.a*r.b*ed.ab.ex*ed.ab.ex/ed.a/ed.b
    ci.non.ex = r.a*ed.ab.non/ed.a + r.b*ed.ab.non/ed.b
    ci.non.non = r.a*ed.ab.non/ed.a + r.b*ed.ab.non/ed.b + r.a*r.b*ed.ab.non*ed.ab.non/ed.a/ed.b
    means.ex.ex = c(means.ex.ex, mean(ci.ex.ex))
    means.ex.non = c(means.ex.non, mean(ci.ex.non))
    hpdis.ex.ex = c(hpdis.ex.ex, HPDI(ci.ex.ex, prob=0.89))
    hpdis.ex.non = c(hpdis.ex.non, HPDI(ci.ex.non, prob=0.89))
    means.non.ex = c(means.non.ex, mean(ci.non.ex))
    means.non.non = c(means.non.non, mean(ci.non.non))
    hpdis.non.ex = c(hpdis.non.ex, HPDI(ci.non.ex, prob=0.89))
    hpdis.non.non = c(hpdis.non.non, HPDI(ci.non.non, prob=0.89))
}

hpdis.ex.ex = matrix(hpdis.ex.ex, nrow=2)
hpdis.ex.non = matrix(hpdis.ex.non, nrow=2)
hpdis.non.ex = matrix(hpdis.non.ex, nrow=2)
hpdis.non.non = matrix(hpdis.non.non, nrow=2)

# pdf("simulated.pdf", width=5, height=4)

plot(means.ex.ex ~ fa.axis, ylim=c(0, 2), type='l', col='orange',
         xlab="Fractional inhibition", ylab="CI", main="Assuming exclusive interaction",
     cex.lab=1.2, cex.axis=1.2, cex.main=1.2, cex.sub=1.2)
lines(means.non.ex ~ fa.axis, col='blue')
shade(hpdis.ex.ex, fa.axis)
shade(hpdis.non.ex, fa.axis)
abline(h=1)
legend(0.2, 2, legend=c("Exclusive", "Nonexclusive"),
       col=c("orange", "blue"), lty=c(1, 1), cex=0.8)

plot(means.ex.non ~ fa.axis, ylim=c(0, 2), type='l', col='orange',
         xlab="Fractional inhibition", ylab="CI", main="Assuming nonexclusive interaction")
lines(means.non.non ~ fa.axis, col='blue')
shade(hpdis.ex.non, fa.axis)
shade(hpdis.non.non, fa.axis)
abline(h=1)

# dev.off()